In [ ]:
import os
import os.path as op
import shutil
import glob
from nilearn import image
from nilearn import masking
import numpy as np
import scipy.stats
import pandas as pd
import datetime
import subprocess

In [ ]:
# Get tasks and cons from your BIDS/code/contrasts.tsv file
# Second column is either 'description' or 'desc' in constrasts.tsv
def get_cons_and_tasks(proj_dir, second_column='description'):
    df = pd.read_csv(proj_dir + "data/BIDS/code/contrasts.tsv", sep="\t")
    cons = [[] for i in range(len(pd.unique(df['task'])))] 

    tasks = []
    for i, row in df.iterrows():
        if row['task'] not in tasks:
            tasks.append(row['task'])
    
    for i, row in df.iterrows():
        cons[tasks.index(row['task'])].append("%s" % (row[second_column].lower()))
    return cons, tasks

In [ ]:
#Edit this line for your project
proj_dir = 'PATH_TO_PROJECT'

cons, tasks = get_cons_and_tasks(proj_dir)
print(type(cons))
#cons = [cons]

# CHANGE TO False IF YOU SET THE USE-ITERATIVE-LEAVE-ONE-OUT FLAG TO FALSE FOR 2ND LEVEL ANALYSES
iterative = True #Did you do iterative subject-level modeling?

tier_proj_dir = proj_dir
tier_dir = op.join(tier_proj_dir,'TIER')
copes_dir = op.join(tier_dir,'original_data','copes')
varcopes_dir = op.join(tier_dir,'original_data','varcopes')
whole_group_design_dir = op.join(tier_dir,'analysis_data','group_analysis')
os.makedirs(whole_group_design_dir, exist_ok = True)

print(cons, tasks)

In [ ]:
for task_idx, task in enumerate(tasks):
#     print("current task: " + tasks[task_idx])
#     print(task_idx)
    
    # subject CSV == list of subj IDs to include in this analysis 
    # THIS EXPECTS YOU TO HAVE PER-TASK CSVs NAMED ACCORDING TO PATTERN BELOW; CSV IN SPECIFIED DIR. 
    # LIST SUBJJECTS IN BIDS FORMAT -- e.g. sub-SAXEEMOfd23 (i.e., with no underscores, with 'sub-' prefix)
    subject_lists = op.join(proj_dir, 'scripts/10_whole_brain_group/subject_lists') 
    subject_file = op.join(subject_lists, 'subjects_task-{}.csv'.format(task))
    participant_IDs = pd.read_csv(subject_file, header=None)
    print(participant_IDs)

    for con in cons[task_idx]:
        print("current contrast: " + con)
        
        cope_file_list = []
        varcope_file_list = []
        
        for sub in np.asarray(participant_IDs): 
            sub = str(sub[0])
            print("current subject: "+ sub)
            if iterative:
                #iterative whole brain
                
                con_img_fname = '{}/{}_{}_fold_{}_exclude_none_con_*_{}_cope.nii.gz'.format(copes_dir, sub, task, '*', con)
                print('CONFILE')
                print(con_img_fname)
                varcope_img_fname = '{}/{}_{}_fold_{}_exclude_none_con_*_{}_varcope.nii.gz'.format(varcopes_dir, sub, task, '*', con)
                print('VARCOPEFILE')
                print(varcope_img_fname)
                
                
                con_matches = glob.glob(con_img_fname)
                varcope_matches = glob.glob(varcope_img_fname)
#                 print(con_matches)
                
                if len(con_matches) > 1 or len(varcope_matches) > 1:
                    print('There are duplicate files present.')
                elif len(con_matches) < 1 or len(varcope_matches) < 1:
                    print(con_img_fname, 'is missing!')
                else:
                    cope_file_list.append(con_matches[0])
                    varcope_file_list.append(varcope_matches[0])
            else:
                # non iterative whole brain
                con_img_fname = '{}/{}_{}_{}_cope.nii.gz'.format(copes_dir, sub, task, con)
                varcope_img_fname = '{}/{}_{}_{}_varcope.nii.gz'.format(varcopes_dir, sub, task, con)

                if not op.isfile(con_img_fname):
                    print(con_img_fname, 'is missing!')
                else:
                    cope_file_list.append(con_img_fname)
                    varcope_file_list.append(varcope_img_fname)

        
        whole_group_outputdir = '{}/randomise_{}_{}'.format(whole_group_design_dir, task, con)
        print('whole group output dir - per task, per contrast')
        print(whole_group_outputdir)
        os.makedirs(whole_group_outputdir, exist_ok = True)
        
        copefile = whole_group_outputdir + '/all_copes.nii.gz'
        print("copes list")
        print(cope_file_list)
        
        
        print('cope file output')
        print(copefile)
        
        cmd = 'fslmerge -t ' + copefile + ' ' 
        cmd = cmd + ' '.join(cope_file_list)
        result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
        print("Merged all copes", result.stdout)
        
        

        varcopefile = whole_group_outputdir + '/all_varcopes.nii.gz'
        
        print("varcopes list")
        print(varcope_file_list)
        
        
        print('varcope file output')
        print(varcopefile)
        
        cmd = 'fslmerge -t ' + varcopefile + ' ' 
        cmd = cmd + ' '.join(varcope_file_list)
        result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
        print("Merged all varcopes", result.stdout)

        
print('All done!')